# 2024 NASACT Training xlwings demo
By: Ohio Auditor of State (Jesse Carroll/Joel Thompson)

## Import Python Libraries

In [1]:
import pandas as pd  #what does 'as' do?
import xlwings as xw  #what is xlwings?
from pathlib import Path
from shutil import copyfile

## Import the data we want to enter into our template

In [3]:
# store path in a variable called current file
current_file = r"random_GL_Reports_2023.xlsx"

# import the GL_Transactions tab from the workbook
df = pd.read_excel(current_file , sheet_name='GL_Transactions')

# show first 5 rows of the dataframe
df.head()

,Fund,FundName,Func,FuncClassName,FuncTypeName,FuncName,Obj,ObjClassName,ObjTypeName,ObjName,...,ExpRevFlag,VoidFlag,PayrollFlag,TitleVIbFlag,AdvTrfFlag,RevAmt,ExpAmt,Unnamed: 30,Unnamed: 31,Link to Table of Contents
0,1,General,1100,Instruction,Regular Instruction,Regular Instruction,411,Purchased Services,Professional/Technical Services,Instruction Services,...,E,NaN,NaN,NaN,NaN,0.0,188.46,NaN,NaN,NaN
1,1,General,1100,Instruction,Regular Instruction,Regular Instruction,411,Purchased Services,Professional/Technical Services,Instruction Services,...,E,NaN,NaN,NaN,NaN,0.0,241.02,NaN,NaN,NaN
2,1,General,1100,Instruction,Regular Instruction,Regular Instruction,411,Purchased Services,Professional/Technical Services,Instruction Services,...,E,NaN,NaN,NaN,NaN,0.0,194.93,NaN,NaN,NaN
3,1,General,1100,Instruction,Regular Instruction,Regular Instruction,411,Purchased Services,Professional/Technical Services,Instruction Services,...,E,NaN,NaN,NaN,NaN,0.0,462.79,NaN,NaN,NaN
4,1,General,1100,Instruction,Regular Instruction,Regular Instruction,411,Purchased Services,Professional/Technical Services,Instruction Services,...,E,NaN,NaN,NaN,NaN,0.0,507.97,NaN,NaN,NaN


### replace fiels with fake data for demo

In [6]:
from faker import Faker
import numpy as np
# Initialize the Faker library
fake = Faker()
df['VendorName'] = df['VendorName'].apply(lambda x: fake.name() if pd.notnull(x) else np.nan)
df['VendorName'].head()

0         Emily Gonzalez
1         Melissa Martin
2         Olivia Barrett
3        Samantha Gibson
4    Christopher Ramirez
Name: VendorName, dtype: object

In [10]:
school_word_list = [
    'student', 'teacher', 'homework', 
    'classroom', 'textbook', 'study', 
    'grade', 'exam', 'subject', 
    'math', 'science', 'history', 
    'english', 'physical education', 'art', 
    'music', 'principal', 'report card', 
    'assignment', 'project', 'quiz', 
    'field trip', 'lunch', 'recess', 
    'library', 'desk', 'chalkboard'
]
fake.sentence(ext_word_list=school_word_list)

'Math chalkboard quiz grade math desk.'

In [13]:
df['Description'].sample(10)

8416         Study science homework library history study.
10965                           Exam project exam student.
13913    Classroom report card homework classroom chalk...
9208        Classroom history quiz recess student history.
19654               Desk recess study principal principal.
17589    Grade math english math assignment textbook li...
21181    English recess exam field trip library princip...
2831                 Exam report card quiz music homework.
21837    Assignment lunch field trip physical education...
22668            Recess math assignment science classroom.
Name: Description, dtype: object

In [12]:
df['Description'] = df['Description'].apply(lambda x: fake.sentence(ext_word_list=school_word_list) if pd.notnull(x) else np.nan)

In [14]:
df.to_excel('random_GL_Reports_2023.xlsx', sheet_name='GL_Transactions', index=False)

### create some variables

In [20]:
#store the current date in a variable called "today"
today = pd.to_datetime('today')

#sort Tran_Date in ascending order
df = df.sort_values(by='TranDate', ascending=True)

# store the first TranDate in a variable called "first_date"
first_date = df['TranDate'].iloc[0]

#print "first date is" and the value of first_date
print(f'first date is {first_date}')

#store the last TranDate in a variable called "last_date"
last_date = df['TranDate'].iloc[-1]

#print "last date is" and the value of last_date
print(f'last date is {last_date}')
print("-"*50)
#total the ExpAmt columns and store in total_exp
total_exp = df['ExpAmt'].sum()
print(f'total expenses: ${total_exp:,.2f}')

#total the RevAmt columns and store in total_rev
total_rev = df['RevAmt'].sum()
print(f'total revenue: ${total_rev:,.2f}')


first date is 2022-07-01 00:00:00
last date is 2023-06-30 00:00:00
--------------------------------------------------
total expenses: $48,319,305.13
total revenue: $50,475,811.17


### entering the variables into the excel template

In [24]:
destination = rf'Results\results.xlsx'
current_template = rf'template.xlsx'

#copy the current_template to the destination
copyfile(current_template, destination)

with xw.App(visible=False) as xl:
    wb = xw.Book(destination)
    xl = xw.apps.active.api

    # add some of our variables from abvoe to tab1 in the workbook
    template_tab = wb.sheets('tab1')
    wb.sheets['tab1'].api.Visible = True
    template_tab.range('B3').value = current_file
    template_tab.range('B5').value = today
    template_tab.range('B7').value = first_date
    template_tab.range('B9').value = last_date
    template_tab.range('B11').value = total_exp
    template_tab.range('B13').value = total_rev

    # # format the cell values/types for the input cells
    # template_tab.range('B5').number_format = "mm/dd/yyyy"
    # template_tab.range('B7').number_format = "mm/dd/yyyy"
    # template_tab.range('B9').number_format = "mm/dd/yyyy"
    # template_tab.range('B11').number_format = "$#,##0.00"
    # template_tab.range('B13').number_format = "$#,##0.00"

    # # add the entire table to a tab called GL_Transactions
    # gl_trans_tab = wb.sheets('GL_Transactions')
    # gl_trans_tab.range('A1').options(index=False).value = df

    # #MORE FORMATTING FUN - remove the comments to see the magic
    # template_tab.range('B:B').autofit()
    # template_tab.range('H:H').autofit()
    # template_tab.range('K:K').autofit()

    # gl_trans_tab.used_range.api.AutoFilter(Field:=1)

    # #bold the text in the first row
    # gl_trans_tab.range('1:1').api.Font.Bold = True

    # #autofit the columns in gl_trans_tab
    # gl_trans_tab.range('A:AD').autofit()

    wb.save(destination)
    wb.close()